In [161]:
import pandas as pd
import numpy as np
import time
import requests
import execjs

In [39]:
# 读取基金文件
manager_history_pd = pd.read_csv('./data/manager/manager_history_pd.csv',low_memory=False)
manager_current_pd = pd.read_csv('./data/manager/manager_current_pd.csv',low_memory=False)
allManager_pd = pd.read_csv('./data/manager/allManager_pd.csv',low_memory=False)


In [106]:
# 计算经理擅长的类型，并保存
allManager_id_s = allManager_pd['manageID']
m_id = []
m_gat = []
for manage_id in allManager_id_s:
    fund_type_history_s = manager_history_pd.loc[manager_history_pd['经理ID'] ==
                                                 manage_id]['基金类型']
    fund_type_current_s = manager_current_pd.loc[manager_current_pd['经理ID'] ==
                                                 manage_id]['基金类型']
    # 合并类型
    fund_type_all_s = fund_type_history_s.append(fund_type_current_s)
#     print(manage_id,fund_type_all_s.shape)
    # 获取类型最多的，作为擅长类型
    if len(fund_type_all_s)  > 0:
        manager_goo_at_type = fund_type_all_s.groupby(
            fund_type_all_s).count().idxmax()
        m_id.append(manage_id)
        m_gat.append(manager_goo_at_type)

manager_goo_at_type_pd = pd.DataFrame(columns=['manageID', 'good_at_type'])
manager_goo_at_type_pd['manageID'] = m_id
manager_goo_at_type_pd['good_at_type'] = m_gat
allManager_good_at_type_pd = pd.merge(
    allManager_pd, manager_goo_at_type_pd, how='left', on='manageID')
allManager_good_at_type_pd.to_csv(
    './data/manager/allManager_pd.csv',
    encoding='utf_8_sig',
    index=False)
allManager_pd = pd.read_csv('./data/manager/allManager_pd.csv',low_memory=False)

2062
2062


In [232]:
# 合并 2 个不同的历史数据，变成新的
manager_history_pd = pd.read_csv('./data/manager/manager_history_pd.csv',low_memory=False)
manager_history_similar_pd = pd.read_csv('./data/manager/manager_history_similar_pd.csv',low_memory=False)
print(manager_history_pd.shape)
print(manager_history_similar_pd.shape)
manager_history_all_pd = pd.merge(
    manager_history_similar_pd,
    manager_history_pd,
    how='left',
    on=['经理ID', '基金代码','基金名称','基金类型','任职天数','任职回报'])

manager_history_all_pd['任职回报'] = manager_history_all_pd['任职回报'].str.strip('%').astype(float)/100
manager_history_all_pd['同类平均'] = manager_history_all_pd['同类平均'].str.strip('%').astype(float)/100
print(manager_history_all_pd.shape)
print(manager_history_all_pd.head())
manager_history_all_pd.to_csv('./data/manager/manager_history_all_pd.csv', index=False, encoding='utf_8_sig')

(19068, 9)
(19118, 10)
(19118, 13)
       经理ID    基金代码         基金名称 基金类型        起始时间 截止时间     任职天数    任职回报  \
0  30198442    5774     华夏产业升级混合  混合型  2018-08-24   至今   1年又28天  0.1533   
1  30198442    5886      华夏鼎沛债券A  债券型  2018-06-26   至今   1年又87天  0.1963   
2  30198442    5887      华夏鼎沛债券C  债券型  2018-06-26   至今   1年又87天  0.1904   
3  30198442  160324  华夏磐晟混合(LOF)  混合型  2018-01-17   至今  1年又247天  0.1142   
4  30198442    2411      华夏新机遇混合  混合型  2017-09-08   至今   2年又13天  0.2328   

     同类平均       同类排名                  相关链接 规模（亿元）             任职时间  
0  0.1703  1339|2757  ['估值图', '基金吧', '档案']   0.49  2018-08-24 ~ 至今  
1  0.0800    21|1260  ['估值图', '基金吧', '档案']   2.04  2018-06-26 ~ 至今  
2  0.0800    24|1260  ['估值图', '基金吧', '档案']   2.33  2018-06-26 ~ 至今  
3  0.0538   594|2311  ['估值图', '基金吧', '档案']   1.75  2018-01-17 ~ 至今  
4  0.1021   342|2255  ['估值图', '基金吧', '档案']   3.11  2017-09-08 ~ 至今  


In [302]:
# 计算经理数据
manager_history_all_pd = pd.read_csv(
    './data/manager/manager_history_all_pd.csv', low_memory=False)


# manager_history_all_pd['任职同类回报比'] = manager_history_all_pd.apply(
#     lambda x: divide(x['任职回报'],x['同类平均']) , axis=1)

manager_history_all_pd['同类排名比'] = manager_history_all_pd.apply(
    lambda x: similarPer(x['同类排名']) , axis=1)

manager_history_all_pd['任职同类差'] = manager_history_all_pd.apply(
    lambda x: x['任职回报'] - x['同类平均'] , axis=1)

manager_history_all_pd['同类差比和'] = manager_history_all_pd.apply(
    lambda x: x['同类排名比'] + x['任职同类差'] , axis=1)

manager_history_all_pd['同类差比乘'] = manager_history_all_pd.apply(
    lambda x: x['同类排名比'] * x['任职同类差'] , axis=1)

manager_history_all_pd['任职天数'] = manager_history_all_pd.apply(
    lambda x: workDay(x['任职天数']) , axis=1)



# manager_history_all_pd.info()
manager_history_all_pd.to_csv('./data/manager/manager_history_all_1_pd.csv', index=False, encoding='utf_8_sig')

In [352]:
# 把个年内的指标放到总表
allManager_pd = pd.read_csv('./data/manager/allManager_pd.csv',low_memory=False)
mID_s = allManager_pd['manageID']
manager_history_all_pd = pd.read_csv(
    './data/manager/manager_history_all_1_pd.csv', low_memory=False)
for y in range(2, 6, 1):
    print("执行到",y,"年")
    m_history_1_list = []
    m_history_2_list = []
    m_history_3_list = []
    m_history_4_list = []
    for mID in mID_s:
        m_history = manager_history_all_pd.loc[manager_history_all_pd['经理ID'] ==
                                               mID]


        m_history_1 = m_history.loc[m_history['任职天数'] > y * 365]['同类排名比'].sum()
        m_history_1_list.append(m_history_1)
        m_history_2 = m_history.loc[m_history['任职天数'] > y * 365]['任职同类差'].sum()
        m_history_2_list.append(m_history_2)
        m_history_3 = m_history.loc[m_history['任职天数'] > y * 365]['同类差比和'].sum()
        m_history_3_list.append(m_history_3)
        m_history_4 = m_history.loc[m_history['任职天数'] > y * 365]['同类差比乘'].sum()
        m_history_4_list.append(m_history_4)

    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内同类排名比',m_history_1_list)
    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内任职同类差',m_history_2_list)
    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内同类差比和',m_history_3_list)
    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内同类差比乘',m_history_4_list)
allManager_pd.to_csv('./data/manager/allManager_1_pd.csv', index=False, encoding='utf_8_sig')

执行到 2 年
执行到 3 年
执行到 4 年
执行到 5 年


In [270]:
# 查找大于 2 年，总差最大的
manager_history_all_pd = pd.read_csv(
    './data/manager/manager_history_all_1_pd.csv', low_memory=False)
# manager_history_all_pd.info()
manager_top_pd = manager_history_all_pd.loc[manager_history_all_pd['任职天数'] > 2* 365]
print(manager_top_pd.head())

        经理ID    基金代码            基金名称  基金类型        起始时间        截止时间  任职天数  \
4   30198442    2411         华夏新机遇混合   混合型  2017-09-08          至今   743   
5   30198442    3300          华夏圆和混合   混合型  2016-12-29          至今   996   
6   30198442       1          华夏成长混合   混合型  2015-01-07          至今  1718   
7   30198442  160311     华夏蓝筹混合(LOF)   混合型  2013-03-11  2017-02-24  1446   
12  30507477  502000  西部利得中证500等权重分级  股票指数  2016-11-24          至今  1031   

      任职回报    同类平均      同类排名                  相关链接 规模（亿元）  \
4   0.2328  0.1021  342|2255  ['估值图', '基金吧', '档案']   3.11   
5   0.1722  0.1914  874|1787  ['估值图', '基金吧', '档案']   5.81   
6   0.1237  0.4737   614|746  ['估值图', '基金吧', '档案']  47.14   
7   0.6583  0.7101   281|550  ['估值图', '基金吧', '档案']  39.36   
12 -0.0036  0.0917   400|632  ['估值图', '基金吧', '档案']   2.32   

                       任职时间   经理任职同类比  经理任职同类差  
4           2017-09-08 ~ 至今  2.280118   0.1307  
5           2016-12-29 ~ 至今  0.899687  -0.0192  
6           2015-01-07 ~ 至今  

In [328]:
# 计算任职时间
def workDay(dayStr):
    manage_wordDay = dayStr.split('又')
    if len(manage_wordDay) > 1:
        manage_word_year = manage_wordDay[0].split('年')[0]
        manage_word_day = manage_wordDay[1].split('天')[0]
        manage_wordDay_Total = int(manage_word_year) * 365 + int(
            manage_word_day)
    else:
        manage_wordDay_Total = dayStr.split('天')[0]
    return manage_wordDay_Total

In [280]:
# 计算相似比率
def similarPer(similarStr):
    x = similarStr.split('|')[0]
    y = similarStr.split('|')[1]
    if x != '-' and y != '-':
        return 1 - int(x)/int(y)
    else:
        return 0
    